# 2.5 自动微分
- 深度学习框架通过自动计算导数，即自动微分（automatic differentiation）来加快求导
- 实际中，根据我们设计的模型，系统会构建一个计算图（computational graph）， 来跟踪计算是哪些数据通过哪些操作组合起来产生输出
## 2.5.1 例子
对函数y=2x<sup>T</sup>x关于列向量x求导

In [1]:
import torch
x=torch.arange(4.0) # create a row vector
x

tensor([0., 1., 2., 3.])

In [2]:
"""
在我们计算y关于x的梯度之前，我们需要一个地方来存储梯度。 重要的是，我们不会在每次对一个参数求导时都分配新的内存。 因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。
注意，一个标量函数关于向量x的梯度是向量，并且与x具有相同的形状。
"""

x.requires_grad=True  # 等价于x=torch.arange(4.0,requires_grad=True)，需要计算梯度
x.grad  # 默认值是None
y=2*torch.dot(x,x)  # 得到了我们赋值给y的标量输出
y


tensor(28., grad_fn=<MulBackward0>)

In [3]:
"""
通过调用反向传播函数来自动计算y关于x每个分量的梯度，并打印这些梯度。
"""
y.backward()
x.grad


tensor([ 0.,  4.,  8., 12.])

In [4]:
"""
验证梯度是否计算正确
"""
x.grad==4*x

tensor([True, True, True, True])

In [5]:
"""
在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
"""
x.grad.zero_()
y = x.sum()
y.backward()
y,x.grad

(tensor(6., grad_fn=<SumBackward0>), tensor([1., 1., 1., 1.]))

## 2.5.2. 非标量变量的反向传播
- 当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。 对于高阶和高维的y和x，求导的结果可以是一个高阶张量。
- 当我们调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
    - 这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

In [6]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
y.sum().backward()  # 等价于y.backward(torch.ones(len(x)))
x.grad

tensor([0., 2., 4., 6.])

## 2.5.3 分离计算
- 将某些计算移动到记录的计算图之外。

In [7]:
"""
下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数。
"""
x.grad.zero_()  # 清0梯度
y = x * x
u = y.detach()  # u不再是关于x的函数，而是一个常数
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [8]:
"""
由于记录了y的计算结果，我们可以随后在y上调用反向传播， 得到y=x*x关于的x的导数，即2*x。
"""
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## 2.5.4. Python控制流的梯度计算
- 使用自动微分的一个好处是： 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。

In [19]:
"""
在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。
"""
def f(a):
    b = a*2
    # print(b)
    while b.norm()<1000:
        b = b*2
        # print(b)
    if b.sum() > 0:
        c=b
    else:
        c = 100*b
    return c

# 计算梯度
a=torch.randn(size=(),requires_grad=True)  # 定义a为一个随机数
d=f(a)
d.backward()
"""
我们现在可以分析上面定义的f函数。 请注意，它在其输入a中是分段线性的。 换言之，对于任何a，存在某个常量标量k，使得f(a)=k*a，其中k的值取决于输入a。 因此，我们可以用d/a验证梯度是否正确。
"""
a,a.grad==d/a

(tensor(-9.0155e-05, requires_grad=True), tensor(False))

## 2.5.5 小结
- 深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上。然后我们记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。